In [3]:
import requests
from bs4 import BeautifulSoup

# URL of the webpage
url = "https://www.calendarr.com/united-kingdom/observances-2024/"

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content of the webpage
soup = BeautifulSoup(response.content, "html.parser")

# List to hold the formatted events
events = []

# Find all the tags that contain class "holiday-month"
months = soup.find_all("span", class_="holiday-month")

# Loop through each month span
for month_tag in months:
    # Get the name of the month
    month = month_tag.text.strip()
    
    # Get the next tags which are expected to contain the holiday information
    next_tags = month_tag.find_all_next("ul", class_="list-holidays")

    # Iterate over each next tag
    for next_tag in next_tags:
        # Check if the tag belongs to the current month
        if next_tag.find_previous("span", class_="holiday-month") != month_tag:
            break

        # Loop through all child list-holiday-box elements within the next_tag
        for box in next_tag.find_all("li", class_="list-holiday-box"):
            # Extract day, weekday, and name from the children tags
            day = box.find("span", class_="holiday-day").text.strip()
            weekday = box.find("span", class_="holiday-weekday").text.strip()
            
            # Event name can be in an <a> or <span> tag with class "holiday-name"
            name_tag = box.find("a", class_="holiday-name") or box.find("span", class_="holiday-name")
            name = name_tag.text.strip() if name_tag else "No name found"

            # Append the formatted event to the list
            events.append(f"{month}, {day}, {weekday}, {name}")

# Print the formatted events
for event in events:
    print(event)


January, 01, Mon, New Year's Day
January, 01, Mon, Mexican Independence Day
January, 01, Mon, Dry January
January, 01, Mon, Veganuary
January, 01, Mon, International Self Care Day
January, 01, Mon, National Bloody Mary Day
January, 02, Tue, World Introvert Day
January, 04, Thu, World Braille Day
January, 04, Thu, National Trivia Day
January, 04, Thu, World Hypnotism Day
January, 05, Fri, National Whipped Cream Day
January, 06, Sat, Epiphany
January, 06, Sat, National Bean Day
January, 07, Sun, Orthodox Christmas
January, 11, Thu, National Milk Day
January, 12, Fri, National Pharmacist Day
January, 14, Sun, World Logic Day
January, 15, Mon, Brew Monday
January, 16, Tue, National Nothing Day
January, 17, Wed, Museum Selfie Day
January, 17, Wed, Guru Gobind Singh Jayanti
January, 18, Thu, National Thesaurus Day
January, 19, Fri, National Popcorn Day
January, 20, Sat, World Religion Day
January, 21, Sun, National Squirrel Appreciation Day
January, 21, Sun, World Snow Day
January, 23, Tue, 

In [1]:
pip install mysql-connector-python

In [6]:
import mysql.connector
from mysql.connector import errorcode

# Function to establish a connection to MySQL
def connect_to_mysql():
    try:
        # Connect to MySQL
        connection = mysql.connector.connect(
            host="", # your database host
            user="", # your database user
            password="", # your database password
            database=""  # your database name
        )
        print("Connected to MySQL!")
        return connection
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("Something is wrong with your username or password")
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("Database does not exist")
        else:
            print(err)

def map_month(month_name):
    months = {
        'January': '01',
        'February': '02',
        'March': '03',
        'April': '04',
        'May': '05',
        'June': '06',
        'July': '07',
        'August': '08',
        'September': '09',
        'October': '10',
        'November': '11',
        'December': '12'
    }
    return months.get(month_name, '01')  # Default to January if not found

# Insert data into MySQL
def insert_data_into_mysql(events):
    connection = connect_to_mysql()
    if connection:
        try:
            cursor = connection.cursor()

            # SQL query to insert data
            insert_query = ("INSERT INTO event "
                            "(description, title, start_date, end_date, created_on, updated_on, image_name, person_id) "
                            "VALUES (%s, %s, %s, %s, NOW(), NOW(), '', 1)")

            # Insert each event into the database
            for event in events:
                event_data = event.split(", ")
                description = "Dummy description"  # You can change this to whatever description you want
                title = event_data[3]  # Event name
                month = map_month(event_data[0])  # Map month name to numerical value
                day = event_data[1]
                start_date = f"2024-{month}-{day}"  # Assuming year is 2024
                end_date = start_date  # Assuming events are single-day
                event_values = (description, title, start_date, end_date)
                cursor.execute(insert_query, event_values)

            # Commit changes
            connection.commit()
            print("Data inserted successfully!")

        except mysql.connector.Error as error:
            print("Failed to insert data into MySQL table:", error)

        finally:
            if connection.is_connected():
                cursor.close()
                connection.close()
                print("MySQL connection is closed")


# Call the function to insert data
insert_data_into_mysql(events)


Connected to MySQL!
Data inserted successfully!
MySQL connection is closed
